# Intact Habitat Cores

This is an indicator for the 2022 base blueprint. We made a slight change in 2023 to make this indicator match our new guidance for zero/nodata value use.

This code just joins the state level ESRI data, which was based on 2011 NLCD, along with other inputs.

Ultimately, we need to either get ESRI to re-run this based on updated NLCD, or we need to re-run all the ESRI state level data based on 2019 NLCD, and any other updated inputs. It is a bear to get all the state level inputs.

Created by Amy Keister, last run by Amy Keister on 23 February, 2022

In [1]:
import os, fnmatch
import arcpy

In [2]:
import time
start = time.time()

In [3]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
SourceWorkspace = r"F:\GIS_DATA\DecisionSupportTools\GreenInfrastructureCenterCores\MapPackageDownloads\ExtractedData"
OutWorkspace = r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\Intact\Intact.gdb"

In [14]:
# define final indicator outputs
Out = r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\Intact\IntactHabitatCores.tif"

In [5]:
# define sub-indicator outputs to help with user support

In [6]:
# define rasters used for cell size, extent, and snapping
Rextent= r"F:\GIS_DATA\SECAS\SE_Blueprint_2022\Southeast_Blueprint_2022_Data_Download\SEBlueprint20221215\Inputs\BaseBlueprint\1_ExtentLayers\BaseBlueprintExtent2022.tif"

In [7]:
# define additional inputs
NLCD= r"F:\GIS_DATA\LanduseLandcover\NLCD\NLCD_landcover_2019_release_all_files_20210604\nlcd_2019_land_cover_l48_20210604\nlcd_2019_land_cover_l48_20210604.img"

### Bring in ESRI cores

In [8]:
# Set the workspace where the downoaded ESRI data is
arcpy.env.workspace = SourceWorkspace

In [9]:
print(arcpy.env.workspace)

F:\GIS_DATA\DecisionSupportTools\GreenInfrastructureCenterCores\MapPackageDownloads\ExtractedData


In [10]:
#look in the source data from ESRI make a list of all the habitat cores
# feature classes
FCList = []

walk = arcpy.da.Walk(SourceWorkspace, datatype="FeatureClass", type="Polygon")

for dirpath, dirnames, filenames in walk:
    for filename in filenames:
        FCList.append(os.path.join(dirpath, filename))

In [11]:
#Print the list made in the above step to see what I found
print(FCList)

['F:\\GIS_DATA\\DecisionSupportTools\\GreenInfrastructureCenterCores\\MapPackageDownloads\\ExtractedData\\AL_GreenInfoData_79175021-AE3D-43CF-856C-09ED583D3240\\v105\\data.gdb\\NHD_AREA', 'F:\\GIS_DATA\\DecisionSupportTools\\GreenInfrastructureCenterCores\\MapPackageDownloads\\ExtractedData\\AL_GreenInfoData_79175021-AE3D-43CF-856C-09ED583D3240\\v105\\data.gdb\\NHD_WATERBODY', 'F:\\GIS_DATA\\DecisionSupportTools\\GreenInfrastructureCenterCores\\MapPackageDownloads\\ExtractedData\\AL_GreenInfoData_79175021-AE3D-43CF-856C-09ED583D3240\\v105\\data.gdb\\NWI_WETLANDS', 'F:\\GIS_DATA\\DecisionSupportTools\\GreenInfrastructureCenterCores\\MapPackageDownloads\\ExtractedData\\AL_GreenInfoData_79175021-AE3D-43CF-856C-09ED583D3240\\v105\\data.gdb\\SSURGO', 'F:\\GIS_DATA\\DecisionSupportTools\\GreenInfrastructureCenterCores\\MapPackageDownloads\\ExtractedData\\AL_GreenInfoData_79175021-AE3D-43CF-856C-09ED583D3240\\v105\\data.gdb\\TNCREGIONS', 'F:\\GIS_DATA\\DecisionSupportTools\\GreenInfrastructur

In [12]:
# too many feature classes! need to narrrow down based on a wildcard to only list the cores
FCListS =[]

for fc in FCList:
    if fnmatch.fnmatch(fc, "*HabitatCores20170305"):
        FCListS.append(fc)

In [13]:
print(FCListS)

['F:\\GIS_DATA\\DecisionSupportTools\\GreenInfrastructureCenterCores\\MapPackageDownloads\\ExtractedData\\AL_GreenInfoData_79175021-AE3D-43CF-856C-09ED583D3240\\v105\\derivedgidata20170305.gdb\\HabitatCores20170305', 'F:\\GIS_DATA\\DecisionSupportTools\\GreenInfrastructureCenterCores\\MapPackageDownloads\\ExtractedData\\AR_GreenInfoData_74AA5169-19E3-4925-96F3-F0A9DBF3656C\\v105\\derivedgidata20170305.gdb\\HabitatCores20170305', 'F:\\GIS_DATA\\DecisionSupportTools\\GreenInfrastructureCenterCores\\MapPackageDownloads\\ExtractedData\\FL_GreenInfoData_63F9EC8E-FA7F-41C3-B677-780AF9A06807\\v105\\derivedgidata20170305.gdb\\HabitatCores20170305', 'F:\\GIS_DATA\\DecisionSupportTools\\GreenInfrastructureCenterCores\\MapPackageDownloads\\ExtractedData\\GA_GreenInfoData_01A434FC-46D2-4C62-ADC3-575DAD67D86D\\v105\\derivedgidata20170305.gdb\\HabitatCores20170305', 'F:\\GIS_DATA\\DecisionSupportTools\\GreenInfrastructureCenterCores\\MapPackageDownloads\\ExtractedData\\IA_GreenInfoData_1EAAADFF-F7D3

### Start Analysis

In [14]:
# Set the workspace where I want the output to go
arcpy.env.workspace = OutWorkspace

In [15]:
print(arcpy.env.workspace)

D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\Intact\Intact.gdb


In [16]:
# Merge all ESRI core state polygons together
arcpy.management.Merge(FCListS, "MergeCore", "", "ADD_SOURCE_INFO")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\Intact\\Intact.gdb\\MergeCore'>

In [17]:
# remove duplicate polygons
arcpy.management.DeleteIdentical("MergeCore", "Value", None, 0)

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\Intact\\Intact.gdb\\MergeCore'>

In [19]:
# convert to raster based on core size acres field
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    arcpy.conversion.PolygonToRaster("MergeCore", "Acres", "HabitatCores", "CELL_CENTER", "NONE", Rextent)

In [20]:
# is null to take NODATA values from core raster and give them a value of 1
out_raster = arcpy.sa.IsNull("HabitatCores"); out_raster.save("HabitatCoresISNull")

In [21]:
# this conditional statements makes makes the NODATA 0
out_raster = arcpy.sa.Con("HabitatCoresISNull", "HabitatCores", 0, "Value = 0"); out_raster.save("HabitatCores0")

In [22]:
# make raster an integer 
# this is only important for back when we were making a continous raster
#out_raster = arcpy.sa.Int("HabitatCores0"); out_raster.save("HabitatCores0Int")

In [23]:
# reclassify

# ArcGIS reclass includes the second value in the class for example,
# 0 1 is <=1 (1 is included)
# 1 5 is >1 - 5 (1 is not included, 5 is included)
# 5 20 is >5 - 20 (5 is not included, 20 is included)

out_raster = arcpy.sa.Reclassify("HabitatCores0", "Value", "0 0 0;1 1000 1;1000 10000 2;10000 9000000 3", "DATA"); out_raster.save("indicato")

In [24]:
# use NLCD extent to remove deep marine
out_raster = arcpy.sa.Con(NLCD, "indicato", '', "Value > 0"); out_raster.save("indicator")

### Finalize indiator

do final steps for all indicators to add description fields, clip and export to SE extent, clip and export to SA extent

In [29]:
# clip to SE 2021 Blueprint extent
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.ExtractByMask("indicator", Rextent); out_raster.save("SEMask")

In [30]:
# export as .tif with SE extent
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    arcpy.management.CopyRaster("SEMask", Out, '', None, "15", "NONE", "NONE", "4_BIT", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

In [16]:
# set code block for next step
codeblock = """
def Reclass(v):
    if v == 3:
        return '3 = Large core (>10,000 acres)'
    elif v == 2:
        return '2 = Medium core (>1,000-10,000 acres)'
    elif v == 1:
        return '1 = Small core (>100-1,000 acres)'
    elif v == 0:
        return '0 = Not a core'
"""

In [17]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField(Out, "descript", "Reclass(!value!)", "PYTHON3", codeblock, "TEXT")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\Intact\\IntactHabitatCores.tif'>

In [18]:
# set code block for next step
codeblock = """
def Reclass1(Value):
	if Value == 3:
		return 0
	if Value == 2:
		return 16
	if Value == 1:
		return 62
	else:
		return 255
		
def Reclass2(Value):
	if Value == 3:
		return 68
	if Value == 2:
		return 123
	if Value == 1:
		return 171
	else:
		return 255
		
def Reclass3(Value):
	if Value == 3:
		return 27
	if Value == 2:
		return 55
	if Value == 1:
		return 114
	else:
		return 255
"""

In [19]:
# calculate Red field
arcpy.management.CalculateField(Out, "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField(Out, "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField(Out, "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\Intact\\IntactHabitatCores.tif'>

In [31]:
end = time.time()
print(end - start)

3313.4667658805847
